Neural Networks


In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import lightgbm as lgb

   
# Read the CSV file into a pandas DataFrame
data = pd.read_csv('C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Data/merged_train_sameday_real.csv')
test_data = pd.read_csv('C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Data/merged_test_sameday_real.csv')
columns_to_remove = ['volume', 'open_interest', 'option_price']
test_data = test_data.drop(columns=columns_to_remove)
data = data.drop(columns=columns_to_remove)
test_data_with_all_predictions = test_data.copy() 

option_columns = [
    'impl_volatility',
    'cp_flag',
    'moneyness',
    'stock_price',
    'strike_price',
    'time_to_expiry',
    'rf',
    'iv_ahbs',
    'iv_ahbs_error',
    'iv_bs',
    'iv_bs_error',  # Adjusted from a duplicate iv_ahbs_error
    'iv_cw',
    'iv_cw_error'
]

option_only = data
print(option_only.head())


   impl_volatility  cp_flag  stock_price  moneyness  time_to_expiry  \
0         0.210270        1      115.545   0.970966              24   
1         0.208124        1      115.545   0.962875              24   
2         0.205474        1      115.545   0.954917              24   
3         0.278442        0      115.545   1.100429              24   
4         0.242212        0      115.545   1.050409              24   

   strike_price     delta     gamma       vega      theta  ...  qmj_safety  \
0         119.0  0.329402  0.057909  10.772000 -16.612410  ...    0.820412   
1         120.0  0.273000  0.053728   9.892975 -15.153390  ...    0.820412   
2         121.0  0.220664  0.048533   8.819048 -13.381580  ...    0.820412   
3         105.0 -0.075725  0.017194   4.246497  -9.138635  ...    0.820412   
4         110.0 -0.186759  0.037192   7.995862 -15.079300  ...    0.820412   

   lag_market_value  row_id  position_in_group   iv_ahbs  iv_ahbs_error  \
0     608960.247515     748  

In [ ]:
###########################################
# PART 1: LIGHTGBM MODEL DEFINITION
###########################################

def create_lgb_model(model_type):

    if model_type == 'LGB1':
        # Standard LightGBM configuration
        params = {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1
        }
    
    elif model_type == 'LGB2':
        # More complex LightGBM configuration with different hyperparameters
        params = {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'num_leaves': 63,
            'learning_rate': 0.01,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.7,
            'bagging_freq': 4,
            'min_data_in_leaf': 20,
            'max_depth': 10,
            'verbose': -1
        }
    
    else:
        raise ValueError("Invalid model type. Choose from 'LGB1' or 'LGB2'.")
    
    return params

def train_and_evaluate_model(params, X_train, y_train, X_test, y_test, num_boost_round=100):

    # Create LightGBM datasets
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    # Train model with early stopping
    model = lgb.train(
        params,
        train_data,
        num_boost_round=num_boost_round,
        valid_sets=[valid_data],
        callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=True)]
    )
    
    # Evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    return model, mse

In [ ]:
###########################################
# PART 2: DATA PREPARATION
###########################################

def prepare_data(option_only):

    # Prepare features and target variables
    feature_columns = [col for col in option_only.columns if col not in 
                      ['iv_bs_error', 'iv_ahbs', "iv_ahbs_error", "iv_bs", 
                       "iv_cw", "iv_cw_error", "impl_volatility"]]
    
    # Instead of using train_test_split, use your predefined sets
    X_train = option_only[feature_columns]  # Features from your training set
    X_test = test_data[feature_columns]  # Features from your test set

    # Target variables for each error type
    y_bs_train = option_only['iv_bs_error']  
    y_bs_test = test_data['iv_bs_error']  

    y_ahbs_train = option_only['iv_ahbs_error']  
    y_ahbs_test = test_data['iv_ahbs_error'] 

    y_cw_train = option_only['iv_cw_error']  
    y_cw_test = test_data['iv_cw_error'] 
    
    # Initialize a StandardScaler to normalize the features
    scaler = StandardScaler()
    
    # Fit the scaler on the training data and transform both training and testing data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create dictionaries to store multiple target variables
    y_train_dict = {
        'bs': y_bs_train,
        'ahbs': y_ahbs_train,
        'cw': y_cw_train
    }
    
    y_test_dict = {
        'bs': y_bs_test,
        'ahbs': y_ahbs_test,
        'cw': y_cw_test
    }
    
    return X_train_scaled, X_test_scaled, y_train_dict, y_test_dict, scaler, feature_columns


In [ ]:
###########################################
# PART 3: PREDICTION FUNCTION
###########################################

def predict_and_add_to_test_data(models, test_data, scaler, feature_columns, error_type='bs'):

    # Create a copy of the test data to avoid modifying the original
    result_df = test_data.copy()
    
    # Extract features from test data
    X_test = test_data[feature_columns].values
    
    # Scale the features using the pre-fitted scaler
    X_test_scaled = scaler.transform(X_test)
    
    # Original value column name
    original_column = f'iv_{error_type}'
    
    # Generate predictions for each model
    for model_name, model in models.items():
        # Make predictions
        predictions = model.predict(X_test_scaled)
        
        # Add predictions to the dataframe
        column_name = f'iv_{error_type}_pred_{model_name}'
        result_df[column_name] = predictions
        
        # Calculate corrected value by adding the error prediction to the original value
        result_df[f'iv_{error_type}_corrected_{model_name}'] = result_df[original_column] - predictions
    
    return result_df

In [ ]:
###########################################
# PART 4: FEATURE IMPORTANCE ANALYSIS
###########################################

def analyze_feature_importance(model, feature_columns):

    # Get feature importance
    importance = model.feature_importance(importance_type='split')
    
    # Create a DataFrame for better visualization
    importance_df = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': importance
    })
    
    # Sort by importance
    importance_df = importance_df.sort_values('Importance', ascending=False)
    
    return importance_df


In [36]:

###########################################
# PART 5: COMPLETE WORKFLOW
###########################################

if __name__ == "__main__":
    # Step 1: Prepare data
    X_train_scaled, X_test_scaled, y_train_dict, y_test_dict, scaler, feature_columns = prepare_data(option_only)
    
    # Step 2: Train models for each target variable
    models = {}
    results = {}
    
    # Dictionary to store all models
    all_models = {
        'bs': {},
        'ahbs': {},
        'cw': {}
    }
    
    # Train models for each error type
    for error_type in ['bs', 'ahbs', 'cw']:
        print(f"\n=== Training models for {error_type} error correction ===")
        
        # Get the appropriate training and test targets
        y_train = y_train_dict[error_type]
        y_test = y_test_dict[error_type]
        
        # Train each model configuration
        for lgb_type in ['LGB1', 'LGB2']:
            model_name = f"{lgb_type}_{error_type}"
            print(f"\nTraining {model_name}...")
            
            # Create model parameters
            params = create_lgb_model(lgb_type)
            
            # Train and evaluate model
            model, mse = train_and_evaluate_model(
                params, X_train_scaled, y_train, X_test_scaled, y_test, num_boost_round=500
            )
            
            # Store model and results
            all_models[error_type][lgb_type] = model
            results[model_name] = {
                'mse': mse,
                'feature_importance': analyze_feature_importance(model, feature_columns)
            }
            
            print(f"{model_name} Test MSE: {mse}")
            
            # Save feature importance
            results[model_name]['feature_importance'].to_csv(f"{model_name}_feature_importance_sameday.csv", index=False)
            
            # Save model (using LightGBM's save_model method)
            model.save_model(f"{model_name}_model_sameday.txt")
    
    # Step 3: Generate predictions on test data for each error type
    for error_type in ['bs', 'ahbs', 'cw']:
        print(f"\n=== Making predictions for {error_type} error correction ===")
        
        # Extract the models for this error type
        current_models = all_models[error_type]
        
        # Make predictions - use the accumulated DataFrame
        test_data_with_all_predictions = predict_and_add_to_test_data(
            current_models, test_data_with_all_predictions, scaler, feature_columns, error_type
        )
        
        # Save intermediate results if desired
        test_data_with_all_predictions.to_csv(f'test_data_with_{error_type}_predictions_sameday.csv', index=False)
        
        # Display sample results from accumulated DataFrame
        print(f"\nSample of test data with {error_type} predictions:")
        display_columns = ['iv_' + error_type]
        for lgb_type in ['LGB1', 'LGB2']:
            display_columns.extend([
                f'iv_{error_type}_pred_{lgb_type}', 
                f'iv_{error_type}_corrected_{lgb_type}'
            ])
        
        print(test_data_with_all_predictions[display_columns].head(5))

    # Final save with all predictions
    test_data_with_all_predictions.to_csv('test_data_with_all_predictions_sameday.csv', index=False)


=== Training models for bs error correction ===

Training LGB1_bs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.000385907
LGB1_bs Test MSE: 0.00038590700819387077

Training LGB2_bs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.00109553
LGB2_bs Test MSE: 0.0010955293396601943

=== Training models for ahbs error correction ===

Training LGB1_ahbs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.000369602
LGB1_ahbs Test MSE: 0.0003696019769994083

Training LGB2_ahbs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.00104573
LGB2_ahbs Test MSE: 0.0010457262525785094

=== Training models for cw error correction ===

Training LGB1_cw...
Training until 

C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Sample of test data with bs predictions:
      iv_bs  iv_bs_pred_LGB1  iv_bs_corrected_LGB1  iv_bs_pred_LGB2  \
0  0.353798         0.149854              0.203944         0.144318   
1  0.353798         0.151848              0.201950         0.147108   
2  0.353798         0.151968              0.201830         0.147984   
3  0.353798         0.156563              0.197236         0.143736   
4  0.353798         0.126640              0.227158         0.128682   

   iv_bs_corrected_LGB2  
0              0.209480  
1              0.206690  
2              0.205814  
3              0.210062  
4              0.225116  

=== Making predictions for ahbs error correction ===


C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Sample of test data with ahbs predictions:
    iv_ahbs  iv_ahbs_pred_LGB1  iv_ahbs_corrected_LGB1  iv_ahbs_pred_LGB2  \
0  0.350970           0.141861                0.209110           0.141294   
1  0.349422           0.143208                0.206213           0.143310   
2  0.348100           0.145342                0.202759           0.144491   
3  0.344389           0.142541                0.201847           0.135025   
4  0.359680           0.126644                0.233036           0.131573   

   iv_ahbs_corrected_LGB2  
0                0.209676  
1                0.206111  
2                0.203609  
3                0.209363  
4                0.228107  

=== Making predictions for cw error correction ===


C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Sample of test data with cw predictions:
      iv_cw  iv_cw_pred_LGB1  iv_cw_corrected_LGB1  iv_cw_pred_LGB2  \
0  0.316475         0.109987              0.206488         0.108184   
1  0.315673         0.110484              0.205189         0.111746   
2  0.315839         0.113152              0.202688         0.113891   
3  0.341157         0.132083              0.209073         0.133381   
4  0.329592         0.092251              0.237341         0.100898   

   iv_cw_corrected_LGB2  
0              0.208291  
1              0.203927  
2              0.201948  
3              0.207775  
4              0.228694  


In [37]:
import os

def load_lgb_models(model_paths_dict):
    loaded_models = {}

    for error_type in model_paths_dict:
        loaded_models[error_type] = {}

        for model_name, path in model_paths_dict[error_type].items():
            try:
                path = path.replace('\\', '/')  # Normalize path
                if not os.path.exists(path):
                    print(f"✗ {error_type} {model_name}: File not found at {path}")
                    loaded_models[error_type][model_name] = None
                    continue

                print(f"Loading {error_type} {model_name}...")
                model = lgb.Booster(model_file=path)
                print(f"✓ Model loaded successfully")

                loaded_models[error_type][model_name] = model

            except Exception as e:
                print(f"✗ Failed to load {error_type} {model_name}: {str(e)}")
                loaded_models[error_type][model_name] = None

    return loaded_models

# Example usage
if __name__ == "__main__":
    # Model paths dictionary with proper path handling for Windows
    # Use forward slashes or raw strings to avoid Unicode escape errors
    model_paths_dict = {
    'bs': {
        'LGB1': r"C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\LightGBM Sameday\Firm Characteristics\LGB1_bs_model_sameday.txt",
        'LGB2': r"C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\LightGBM Sameday\Firm Characteristics\LGB2_bs_model_sameday.txt"
    },
    'ahbs': {
        'LGB1': r"C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\LightGBM Sameday\Firm Characteristics\LGB1_ahbs_model_sameday.txt",
        'LGB2': r"C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\LightGBM Sameday\Firm Characteristics\LGB2_ahbs_model_sameday.txt"
    },
    'cw': {
        'LGB1': r"C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\LightGBM Sameday\Firm Characteristics\LGB1_cw_model_sameday.txt",
        'LGB2': r"C:\Users\maxva\OneDrive - Tilburg University\Msc. Data Science\Master Thesis\Code\Models\LightGBM Sameday\Firm Characteristics\LGB2_cw_model_sameday.txt"
    }
    }

    
    # Load all models
    loaded_models = load_lgb_models(model_paths_dict)
    
    # Print summary of loaded models
    print("\n=== Model Loading Summary ===")
    for error_type in loaded_models:
        for model_name in loaded_models[error_type]:
            status = "✓ Loaded" if loaded_models[error_type][model_name] is not None else "✗ Failed"
            print(f"{error_type} {model_name}: {status}")

Loading bs LGB1...
✓ Model loaded successfully
Loading bs LGB2...
✓ Model loaded successfully
Loading ahbs LGB1...
✓ Model loaded successfully
Loading ahbs LGB2...
✓ Model loaded successfully
Loading cw LGB1...
✓ Model loaded successfully
Loading cw LGB2...
✓ Model loaded successfully

=== Model Loading Summary ===
bs LGB1: ✓ Loaded
bs LGB2: ✓ Loaded
ahbs LGB1: ✓ Loaded
ahbs LGB2: ✓ Loaded
cw LGB1: ✓ Loaded
cw LGB2: ✓ Loaded


In [38]:

# 2) Define features (remove target + metadata columns)
feature_columns = [col for col in option_only.columns if col not in 
                    ['iv_bs_error', 'iv_ahbs', "iv_ahbs_error", "iv_bs", 
                    "iv_cw", "iv_cw_error", "impl_volatility"]]
print(f"Feature columns: {feature_columns}")
# 3) Add model predictions and corrections
df = option_only
for error_type in ['bs', 'ahbs', 'cw']:
    print(f"\n-- Predicting corrections for {error_type.upper()} --")
    
    models_for_type = loaded_models[error_type]  # {'LGB1': model_obj, 'NN4': model_obj}
    
    # This helper must add pred/corrected cols in-place
    df = predict_and_add_to_test_data(
        models=models_for_type,
        test_data=df,
        scaler=scaler,
        feature_columns=feature_columns,
        error_type=error_type
    )
    
    # Preview new prediction columns
    cols = ['impl_volatility', f'iv_{error_type}']
    for model in ['LGB1', 'LGB2']:
        cols += [f'iv_{error_type}_pred_{model}', f'iv_{error_type}_corrected_{model}']
    print(df[cols].head(3))



Feature columns: ['cp_flag', 'stock_price', 'moneyness', 'time_to_expiry', 'strike_price', 'delta', 'gamma', 'vega', 'theta', 'rf', 'me', 'ff49', 'div12m_me', 'chcsho_12m', 'eqnpo_12m', 'ret_1_0', 'ret_3_1', 'ret_6_1', 'ret_9_1', 'ret_12_1', 'ret_12_7', 'ret_60_12', 'seas_1_1an', 'seas_1_1na', 'seas_2_5an', 'seas_2_5na', 'seas_6_10an', 'seas_6_10na', 'seas_11_15an', 'seas_11_15na', 'at_gr1', 'sale_gr1', 'capx_gr1', 'inv_gr1', 'debt_gr3', 'sale_gr3', 'capx_gr3', 'inv_gr1a', 'lti_gr1a', 'sti_gr1a', 'coa_gr1a', 'col_gr1a', 'cowc_gr1a', 'ncoa_gr1a', 'ncol_gr1a', 'nncoa_gr1a', 'fnl_gr1a', 'nfna_gr1a', 'tax_gr1a', 'be_gr1a', 'ebit_sale', 'gp_at', 'cop_at', 'ope_be', 'ni_be', 'ebit_bev', 'netis_at', 'eqnetis_at', 'dbnetis_at', 'oaccruals_at', 'oaccruals_ni', 'taccruals_at', 'taccruals_ni', 'noa_at', 'opex_at', 'at_turnover', 'sale_bev', 'cash_at', 'sale_emp_gr1', 'emp_gr1', 'ni_inc8q', 'noa_gr1a', 'ppeinv_gr1a', 'lnoa_gr1a', 'capx_gr2', 'saleq_gr1', 'niq_be', 'niq_at', 'niq_be_chg1', 'niq_at_

C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


   impl_volatility     iv_bs  iv_bs_pred_LGB1  iv_bs_corrected_LGB1  \
0         0.210270  0.353798         0.153442              0.200356   
1         0.208124  0.353798         0.153613              0.200185   
2         0.205474  0.353798         0.157860              0.195938   

   iv_bs_pred_LGB2  iv_bs_corrected_LGB2  
0         0.150535              0.203263  
1         0.153409              0.200389  
2         0.154192              0.199606  

-- Predicting corrections for AHBS --


C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


   impl_volatility   iv_ahbs  iv_ahbs_pred_LGB1  iv_ahbs_corrected_LGB1  \
0         0.210270  0.346996           0.145659                0.201338   
1         0.208124  0.346099           0.145598                0.200501   
2         0.205474  0.345399           0.143739                0.201660   

   iv_ahbs_pred_LGB2  iv_ahbs_corrected_LGB2  
0           0.145400                0.201596  
1           0.146362                0.199738  
2           0.148304                0.197095  

-- Predicting corrections for CW --


C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


   impl_volatility     iv_cw  iv_cw_pred_LGB1  iv_cw_corrected_LGB1  \
0         0.210270  0.316948         0.114489              0.202460   
1         0.208124  0.318967         0.118588              0.200379   
2         0.205474  0.321854         0.117341              0.204514   

   iv_cw_pred_LGB2  iv_cw_corrected_LGB2  
0         0.115532              0.201417  
1         0.120524              0.198443  
2         0.119770              0.202084  


Calculate IVRMSE

In [ ]:

def calculate_ivrmse(predictions_df, error_types=['bs', 'ahbs', 'cw'], models=['LGB1', 'LGB2']):

    results = {}
    
    # Calculate IVRMSE for each error type and model
    for error_type in error_types:
        orig_col = f'iv_{error_type}'
        
        # Calculate base IVRMSE (before correction)
        base_rmse = np.sqrt(mean_squared_error(predictions_df['impl_volatility'], predictions_df[orig_col]))
        results[f"{error_type}_base"] = base_rmse
        
        # Calculate IVRMSE for each model
        for model in models:
            corrected_col = f'iv_{error_type}_corrected_{model}'
            
            # Skip if corrected column doesn't exist
            if corrected_col not in predictions_df.columns:
                print(f"Warning: {corrected_col} column not found, skipping...")
                continue
                
            # Calculate IVRMSE for the corrected predictions
            corrected_rmse = np.sqrt(mean_squared_error(predictions_df['impl_volatility'], predictions_df[corrected_col]))
            results[f"{error_type}_{model}"] = corrected_rmse
            
            # Calculate improvement percentage
            improvement = (base_rmse - corrected_rmse) / base_rmse * 100
            results[f"{error_type}_{model}_improvement"] = improvement
    
    return results

# Example usage
if __name__ == "__main__":
    # Load the predictions DataFrame (assuming it was saved in the main script)
    test_data_with_all_predictions = pd.read_csv('test_data_with_all_predictions_sameday.csv')
    
    # Calculate IVRMSE
    ivrmse_results = calculate_ivrmse(test_data_with_all_predictions)
    
    # Print results in a table format
    print("\n=== IVRMSE Results ===")
    print(f"{'Model':<15} {'IVRMSE':<10} {'Improvement':<12}")
    print("-" * 40)
    
    for error_type in ['bs', 'ahbs', 'cw']:
        base_key = f"{error_type}_base"
        if base_key in ivrmse_results:
            base_rmse = ivrmse_results[base_key]
            print(f"{error_type.upper():<15} {base_rmse:.6f}  {'(baseline)':<12}")
            
            for model in ['LGB1', 'LGB2']:
                model_key = f"{error_type}_{model}"
                imp_key = f"{error_type}_{model}_improvement"
                
                if model_key in ivrmse_results:
                    print(f"{model_key:<15} {ivrmse_results[model_key]:.6f}  {ivrmse_results[imp_key]:.2f}%")
            print("-" * 40)
    
    # Find best overall model
    model_keys = [k for k in ivrmse_results.keys() if not k.endswith('base') and not k.endswith('improvement')]
    if model_keys:
        best_model = min(model_keys, key=lambda k: ivrmse_results[k])
        print(f"\nBest overall model: {best_model} with IVRMSE = {ivrmse_results[best_model]:.6f}")
        
        base_key = f"{best_model.split('_')[0]}_base"
        imp_key = f"{best_model}_improvement"
        if base_key in ivrmse_results and imp_key in ivrmse_results:
            print(f"Improvement over baseline: {ivrmse_results[imp_key]:.2f}%")
            
    # Export results to CSV for further analysis
    results_df = pd.DataFrame({
        'Model': list(ivrmse_results.keys()),
        'Value': list(ivrmse_results.values())
    })
    results_df.to_csv('ivrmse_results_lightgbm_sameday.csv', index=False)


=== IVRMSE Results ===
Model           IVRMSE     Improvement 
----------------------------------------
BS              0.150568  (baseline)  
bs_LGB1         0.019645  86.95%
bs_LGB2         0.033099  78.02%
----------------------------------------
AHBS            0.134068  (baseline)  
ahbs_LGB1       0.019225  85.66%
ahbs_LGB2       0.032338  75.88%
----------------------------------------
CW              0.128764  (baseline)  
cw_LGB1         0.019425  84.91%
cw_LGB2         0.030454  76.35%
----------------------------------------

Best overall model: ahbs_LGB1 with IVRMSE = 0.019225
Improvement over baseline: 85.66%


In [ ]:

# Define the moneyness groups
moneyness_groups = ['DOTMC', 'OTMC', 'ATM', 'OTMP', 'DOTMP']

def analyze_ivrmse_by_moneyness(test_data):

    # 1. Group the test data by moneyness
    results = {}
    for group in moneyness_groups:
        group_data = test_data[test_data['moneyness_category'] == group]
        
        # Skip if no data in this group
        if len(group_data) == 0:
            print(f"Warning: No data found for moneyness group {group}")
            continue
            
        # Initialize results for this group
        results[group] = {}
        
        # 2. Calculate IVRMSE for each error type and model
        for error_type in ['bs', 'ahbs', 'cw']:
            # Column names
            orig_col = f'iv_{error_type}'
            
            # Skip if column doesn't exist
            if orig_col not in group_data.columns:
                print(f"Warning: Column {orig_col} not found, skipping...")
                continue
                
            # Calculate baseline IVRMSE
            base_rmse = np.sqrt(mean_squared_error(
                group_data['impl_volatility'], 
                group_data[orig_col]
            ))
            
            # Store baseline IVRMSE
            results[group][f"{error_type}_base"] = base_rmse
            
            # Calculate corrected IVRMSE for each model
            for model in ['LGB1', 'LGB2']:
                corrected_col = f'iv_{error_type}_corrected_{model}'
                
                # Skip if column doesn't exist
                if corrected_col not in group_data.columns:
                    print(f"Warning: Column {corrected_col} not found, skipping...")
                    continue
                    
                corrected_rmse = np.sqrt(mean_squared_error(
                    group_data['impl_volatility'], 
                    group_data[corrected_col]
                ))
                
                # Calculate improvement percentage
                improvement = (base_rmse - corrected_rmse) / base_rmse * 100
                
                # Store results
                results[group][f"{error_type}_{model}"] = corrected_rmse
                results[group][f"{error_type}_{model}_improvement"] = improvement
    
    return results

def format_results_table(results):

    # Prepare IVRMSE table data
    ivrmse_data = []
    for group, group_results in results.items():
        row = {'Moneyness Group': group}
        
        for key, value in group_results.items():
            if not key.endswith('improvement'):
                if key.endswith('base'):
                    # Format baseline columns
                    error_type = key.split('_')[0].upper()
                    row[f"{error_type} Base"] = value
                else:
                    # Format model columns
                    parts = key.split('_')
                    error_type = parts[0].upper()
                    model = parts[1]
                    row[f"{error_type} {model}"] = value
        
        ivrmse_data.append(row)
    
    # Prepare improvement percentage table data
    improvement_data = []
    for group, group_results in results.items():
        row = {'Moneyness Group': group}
        
        for key, value in group_results.items():
            if key.endswith('improvement'):
                # Format improvement columns
                parts = key.replace('_improvement', '').split('_')
                error_type = parts[0].upper()
                model = parts[1]
                row[f"{error_type} {model}"] = value
        
        improvement_data.append(row)
    
    # Create DataFrames
    ivrmse_df = pd.DataFrame(ivrmse_data)
    improvement_df = pd.DataFrame(improvement_data)
    
    # Sort columns for better readability
    ivrmse_cols = ['Moneyness Group']
    improvement_cols = ['Moneyness Group']
    
    for et in ['BS', 'AHBS', 'CW']:
        ivrmse_cols.extend([f"{et} Base", f"{et} LGB1", f"{et} LGB2"])
        improvement_cols.extend([f"{et} LGB1", f"{et} LGB2"])
    
    # Reorder columns if they exist
    ivrmse_df = ivrmse_df[[col for col in ivrmse_cols if col in ivrmse_df.columns]]
    improvement_df = improvement_df[[col for col in improvement_cols if col in improvement_df.columns]]
    
    return ivrmse_df, improvement_df

def find_best_models(results):

    best_models = []
    
    for group, group_results in results.items():
        for error_type in ['bs', 'ahbs', 'cw']:
            # Get baseline IVRMSE
            base_key = f"{error_type}_base"
            if base_key not in group_results:
                continue
                
            base_rmse = group_results[base_key]
            
            # Find best model for this error type
            models = [m for m in ['LGB1', 'LGB2'] if f"{error_type}_{m}" in group_results]
            if not models:
                continue
            
            # Find model with lowest IVRMSE    
            best_model = min(models, key=lambda m: group_results[f"{error_type}_{m}"])
            best_rmse = group_results[f"{error_type}_{best_model}"]
            improvement = group_results[f"{error_type}_{best_model}_improvement"]
            
            # Also get the values for the other model for comparison
            other_models = [m for m in models if m != best_model]
            other_model_data = {}
            if other_models:
                other_model = other_models[0]
                other_rmse = group_results[f"{error_type}_{other_model}"]
                other_improvement = group_results[f"{error_type}_{other_model}_improvement"]
                other_model_data = {
                    f'Other Model': other_model,
                    f'Other IVRMSE': other_rmse,
                    f'Other Improvement %': other_improvement
                }
            
            model_data = {
                'Moneyness Group': group,
                'Error Type': error_type.upper(),
                'Best Model': best_model,
                'Base IVRMSE': base_rmse,
                'Best IVRMSE': best_rmse,
                'Improvement %': improvement
            }
            
            # Add other model data if available
            model_data.update(other_model_data)
            
            best_models.append(model_data)
    
    return pd.DataFrame(best_models)

def print_formatted_tables(results):

    # Format results into DataFrames
    ivrmse_df, improvement_df = format_results_table(results)
    
    # Format IVRMSE table
    pd.set_option('display.float_format', '{:.6f}'.format)
    print("=" * 80)
    print("IVRMSE by Moneyness Group (Sameday)")
    print("=" * 80)
    print(ivrmse_df.to_string(index=False))
    
    # Format improvement table
    pd.set_option('display.float_format', '{:.2f}%'.format)
    print("\n" + "=" * 80)
    print("Improvement Percentage by Moneyness Group")
    print("=" * 80)
    print(improvement_df.to_string(index=False))
    
    # Find best models
    best_models_df = find_best_models(results)
    
    # Reset float format for mixed table
    pd.set_option('display.float_format', None)
    print("\n" + "=" * 80)
    print("Best Model by Moneyness Group and Error Type")
    print("=" * 80)
    # Format specific columns
    best_models_df['Base IVRMSE'] = best_models_df['Base IVRMSE'].map('{:.6f}'.format)
    best_models_df['Best IVRMSE'] = best_models_df['Best IVRMSE'].map('{:.6f}'.format)
    best_models_df['Improvement %'] = best_models_df['Improvement %'].map('{:.2f}%'.format)
    print(best_models_df.to_string(index=False))
    
    # Summary statistics
    print("\n" + "=" * 80)
    print("Summary Statistics")
    print("=" * 80)
    
    # Count best model occurrences
    model_counts = best_models_df['Best Model'].value_counts()
    print(f"Overall best model distribution: {dict(model_counts)}")
    
    # Average improvement by moneyness group
    print("\nAverage improvement by moneyness group:")
    # Convert percentage strings to numeric values
    best_models_df['Improvement_Numeric'] = pd.to_numeric(best_models_df['Improvement %'].str.rstrip('%'))
    
    # Group and calculate means
    avg_improvement = best_models_df.groupby('Moneyness Group')['Improvement_Numeric'].mean()
    
    # Handle the case where there's only one group (which returns a scalar)
    if isinstance(avg_improvement, pd.Series):
        # Sort if it's a Series with multiple values
        sorted_improvements = avg_improvement.sort_values(ascending=False)
        for group, imp in sorted_improvements.items():
            print(f"  {group}: {imp:.2f}%")
    else:
        # Just print the single value if it's a scalar
        group = best_models_df['Moneyness Group'].iloc[0]
        print(f"  {group}: {avg_improvement:.2f}%")
    
    # Save results to CSV files
    ivrmse_df.to_csv('ivrmse_by_moneyness_lightgbm.csv', index=False)
    improvement_df.to_csv('improvement_by_moneyness_lightgbm.csv', index=False)
    best_models_df.to_csv('best_models_by_moneyness_lightgbm.csv', index=False)

if __name__ == "__main__":
    # Load test data with predictions
    try:
        test_data_with_all_predictions = pd.read_csv('test_data_with_all_predictions_sameday.csv')
        
        # Check if 'moneyness_category' exists, if not, create it
        if 'moneyness_category' not in test_data_with_all_predictions.columns:
            print("Creating moneyness categories...")
            # Define moneyness boundaries
            test_data_with_all_predictions['moneyness_category'] = pd.cut(
                test_data_with_all_predictions['moneyness'],
                bins=[-float('inf'), 0.9, 0.97, 1.03, 1.1, float('inf')],
                labels=moneyness_groups
            )
        
        # Run analysis
        results = analyze_ivrmse_by_moneyness(test_data_with_all_predictions)
        print_formatted_tables(results)
        
    except FileNotFoundError:
        print("Error: Test data file not found. Please run the main script first.")

IVRMSE by Moneyness Group (Sameday)
Moneyness Group  BS Base  BS LGB1  BS LGB2  AHBS Base  AHBS LGB1  AHBS LGB2  CW Base  CW LGB1  CW LGB2
          DOTMC 0.135785 0.022207 0.036868   0.135335   0.021645   0.035377 0.122504 0.021701 0.030282
           OTMC 0.134271 0.017324 0.030313   0.126841   0.016992   0.029372 0.125176 0.017629 0.028439
            ATM 0.140269 0.017199 0.029902   0.135411   0.017170   0.029854 0.131622 0.017457 0.030061
           OTMP 0.131156 0.017339 0.030115   0.129842   0.017238   0.030923 0.128338 0.017570 0.030008
          DOTMP 0.195506 0.023727 0.038147   0.142425   0.022836   0.036510 0.134354 0.022623 0.033389

Improvement Percentage by Moneyness Group
Moneyness Group  BS LGB1  BS LGB2  AHBS LGB1  AHBS LGB2  CW LGB1  CW LGB2
          DOTMC   83.65%   72.85%     84.01%     73.86%   82.29%   75.28%
           OTMC   87.10%   77.42%     86.60%     76.84%   85.92%   77.28%
            ATM   87.74%   78.68%     87.32%     77.95%   86.74%   77.16%
       